# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298362222354                   -0.85    5.2
  2   -8.300221329062       -2.73       -1.25    1.0
  3   -8.300447150112       -3.65       -1.89    3.8
  4   -8.300461594353       -4.84       -2.76    1.4
  5   -8.300463904197       -5.64       -3.02    9.5
  6   -8.300464269755       -6.44       -3.22    1.0
  7   -8.300464453521       -6.74       -3.36    2.2
  8   -8.300464547297       -7.03       -3.48    1.2
  9   -8.300464622663       -7.12       -3.70    1.0
 10   -8.300464634996       -7.91       -3.86    1.0
 11   -8.300464643352       -8.08       -4.17    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63688613853                   -0.71    5.1
  2   -16.64112997449       -2.37       -1.15    1.0
  3   -16.67891637054       -1.42       -1.75    4.5
  4   -16.67920082091       -3.55       -2.33    4.1
  5   -16.67924618674       -4.34       -2.81    2.8
  6   -16.67928244401       -4.44       -3.21    4.6
  7   -16.67928464117       -5.66       -3.36    2.6
  8   -16.67928618062       -5.81       -4.13    2.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32636542308                   -0.56    7.6
  2   -33.33426206452       -2.10       -1.00    1.2
  3   -33.33670335657       -2.61       -1.71    5.6
  4   -33.33692161823       -3.66       -2.62    6.9
  5   -33.33694311838       -4.67       -3.11    7.6
  6   -33.33694386322       -6.13       -3.94    4.2
  7   -33.33694392630       -7.20       -4.50    6.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298346178195                   -0.85    4.8
  2   -8.300247929665       -2.72       -1.59    1.0
  3   -8.300436769972       -3.72       -2.45    5.6
  4   -8.300336097040   +   -4.00       -2.20    9.1
  5   -8.300464411314       -3.89       -3.60    3.2
  6   -8.300464591014       -6.75       -3.93    2.0
  7   -8.300464631091       -7.40       -4.21    1.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32512303679                   -0.56    6.8
  2   -33.15576357165   +   -0.77       -1.19    1.1
  3   +84.82042200232   +    2.07       -0.03   15.1
  4   -33.01605442567        2.07       -1.20    8.2
  5   -31.97838279961   +    0.02       -0.98    5.6
  6   -32.01596480501       -1.43       -0.98    5.2
  7   -33.32974240187        0.12       -1.92    4.0
  8   -33.33424809332       -2.35       -2.18    2.8
  9   -33.33492194942       -3.17       -2.10    2.8
 10   -33.33650269906       -2.80       -2.40    2.2
 11   -33.33693103019       -3.37       -3.12    2.2
 12   -33.33693925391       -5.08       -3.29    3.0
 13   -33.33693803084   +   -5.91       -3.44    2.6
 14   -33.33694288534       -5.31       -3.98    2.0
 15   -33.33694384532       -6.02       -4.48    3.5


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.